In [1]:

from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.optimizers import SGD

import numpy as np

Using TensorFlow backend.


In [2]:
X_train = np.load("X_train.dat")
y_train = np.load("y_train.dat")

In [3]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train, random_state=0)

In [4]:
X_train = X_train/255

print(X_train.shape)
print(y_train.shape)

# split data into validation set
X_train_validation = X_train[700:, :,:,:]
X_train = X_train[:700, :, :, :]
y_train_validation = y_train[700:, :]
y_train = y_train[:700,:]
X_train = X_train[:700, :, :, :]
print("Validation x train set:" + str(X_train_validation.shape))
print("X train set:" + str(X_train.shape))
print("Validation y train set:" + str(y_train_validation.shape))
print("Y train set:" + str(y_train.shape))

(900, 124, 174, 3)
(900, 10)
Validation x train set:(200, 124, 174, 3)
X train set:(700, 124, 174, 3)
Validation y train set:(200, 10)
Y train set:(700, 10)


## Build the CNN network

In [7]:
model = Sequential()
# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(32, (7, 7), activation='relu', input_shape=X_train.shape[1:]))
model.add(BatchNormalization())
model.add(Conv2D(32, (7, 7), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(10, activation='softmax'))

## Begin training the model

In [ ]:
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile('adam', 'categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=25)
model.save("trained_music_classifier.h5")

Epoch 1/25
700/700 [==============================] - 366s - loss: 13.2252 - acc: 0.1471    
Epoch 2/25
700/700 [==============================] - 411s - loss: 13.9767 - acc: 0.1329    
Epoch 3/25
700/700 [==============================] - 365s - loss: 13.6774 - acc: 0.1514    
Epoch 4/25
700/700 [==============================] - 386s - loss: 13.6083 - acc: 0.1557    
Epoch 5/25
700/700 [==============================] - 356s - loss: 13.5172 - acc: 0.1600    
Epoch 6/25
700/700 [==============================] - 343s - loss: 14.5754 - acc: 0.0957    
Epoch 7/25
700/700 [==============================] - 342s - loss: 14.9622 - acc: 0.0700    
Epoch 8/25
700/700 [==============================] - 344s - loss: 13.9419 - acc: 0.1329    
Epoch 9/25
700/700 [==============================] - 343s - loss: 13.7234 - acc: 0.1486    
Epoch 10/25
700/700 [==============================] - 342s - loss: 13.7925 - acc: 0.1443    
Epoch 11/25
700/700 [==============================] - 335s - loss: 1

## Evaluate the model

In [ ]:
score = model.evaluate(X_train_validation, y_train_validation, batch_size=32, verbose=1)
print(score)